In [7]:
import numpy as np
import theano
import nnb

In [21]:
from nltk.tokenize import word_tokenize
from __future__ import print_function
import codecs

In [40]:
from chios import feats_glove

glove = feats_glove._get_glove_dict('data/glove.6B.50d.txt')

In [86]:
np.zeros(50)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [106]:
import csv

def glove_bow(sent):
    tokens = word_tokenize(sent)
    gtokens = [glove[t] for t in tokens if t in glove]
    if not gtokens:
        return np.zeros(50)
    return np.array(gtokens).mean(axis=0)

def load_sents(fname):
    labels = []
    sent0 = []
    sent1 = []
    sent2 = []
    i = 0
    with open(fname) as f:
        c = csv.DictReader(f)
        for l in c:
            labels.append(int(l['isCorrect']))
            sent0.append(l['qText'].decode('utf8'))
            sent1.append(l['aText'].decode('utf8'))
            sent2.append(l['abstract'].decode('utf8'))
            print('%d\r', i, end='')
            i += 1
    return (np.array([glove_bow(s) for s in sent0]), np.array([glove_bow(s) for s in sent1]), np.array([glove_bow(s) for s in sent2]), np.array(labels, dtype='int32'))

In [107]:
s0, s1, s2, y = load_sents('solraoccur-ck12.csv')

 4825

In [91]:
np.size(y[y==1])

1261

In [92]:
np.size(s0, axis=1)

50

In [54]:
glove_bow('baa')

array([  1.28560000e-03,  -7.31950000e-01,  -7.92120000e-01,
         3.21430000e-01,  -2.08460000e-01,   2.82520000e-01,
        -7.11100000e-01,  -6.67910000e-01,   6.36290000e-01,
        -5.54920000e-01,   1.38270000e+00,   8.31750000e-01,
        -7.34270000e-01,  -9.98490000e-02,   4.76990000e-01,
         1.28190000e+00,   5.22030000e-01,   4.08540000e-01,
        -1.53810000e+00,  -2.96910000e-02,   7.83350000e-01,
         9.27590000e-02,  -8.32810000e-01,   7.86090000e-01,
        -8.46890000e-01,  -3.57790000e-01,  -9.69560000e-02,
         4.55210000e-01,  -3.39880000e-01,  -2.53210000e-01,
        -5.62430000e-01,   9.33300000e-01,   2.51210000e-01,
         1.43530000e+00,   1.44480000e-01,  -1.15210000e+00,
         5.51970000e-01,  -1.26010000e+00,  -1.33320000e-01,
         6.75440000e-01,   4.62340000e-01,   1.60380000e-02,
         5.56810000e-01,   5.20990000e-01,  -1.08640000e+00,
        -3.56790000e-02,  -4.70180000e-01,   1.29270000e+00,
         3.67500000e-01,

In [227]:
# L is 3*100 vector with s0, s1, s2 embeddings
L0 = nnb.InputLayer(ndim=1)
L1 = nnb.InputLayer(ndim=1)
L2 = nnb.InputLayer(ndim=1)
C01 = nnb.ConcatenationModel()
CC = nnb.ConcatenationModel()
C1 = nnb.PerceptronLayer(insize=2*50, outsize=50)
C2 = nnb.PerceptronLayer(insize=2*50, outsize=1)
qajoin = (L0 & L1) | C01 | C1
model = (qajoin & L2) | CC | C2

In [228]:
expected_output = nnb.InputLayer(ndim=1, dtype='int32')
cost_func = nnb.cost.CrossEntropyError()
network_cost = (model & expected_output) | cost_func

In [229]:
trainer = nnb.train.SGDTrainer(model=network_cost, L2_reg=1.0)

In [203]:
tdata = [[s0[i], s1[i], s2[i], np.array([y[i]*2-1], dtype='int32')] for i in range(s0.shape[0])]

In [230]:
trainer.train(tdata)

In [231]:
ff = model.compile()

In [234]:
for i in range(10):
    print(ff(s0[i], s1[i], s2[i]), tdata[i][3])

[ 0.49337077] [-1]
[ 0.49477929] [1]
[ 0.48821554] [-1]
[ 0.48910489] [-1]
[ 0.48736033] [-1]
[ 0.48843973] [-1]
[ 0.48618153] [1]
[ 0.49131534] [-1]
[ 0.48764079] [1]
[ 0.49031397] [-1]


In [233]:
C2.params[0].get_value()

array([[ -9.12021935e-03],
       [ -1.29996386e-02],
       [ -1.57607323e-02],
       [ -1.40582543e-03],
       [ -9.42084621e-03],
       [  1.22101799e-02],
       [ -1.96370770e-02],
       [  1.70314188e-02],
       [  8.27188316e-03],
       [  3.30089549e-04],
       [ -1.56943603e-02],
       [ -1.41518308e-03],
       [  9.93443589e-03],
       [ -4.80995374e-03],
       [ -1.60372509e-02],
       [ -1.12048697e-03],
       [ -4.44293435e-03],
       [  1.77180542e-02],
       [  5.94545204e-03],
       [ -8.77283844e-03],
       [  1.11316889e-02],
       [ -2.31070905e-02],
       [ -2.59734235e-03],
       [ -4.09359793e-05],
       [ -7.89427744e-03],
       [  2.23419664e-03],
       [ -3.12076876e-04],
       [ -3.41965525e-04],
       [  3.83875884e-03],
       [  1.39420802e-02],
       [  6.53064716e-03],
       [  1.51883064e-03],
       [ -6.35162229e-03],
       [  1.04885354e-04],
       [  2.46474871e-03],
       [  2.53464432e-03],
       [  9.91739383e-03],
 